SQLAlchemy

#1
SQLAlchemy is a toolkit for working with relational databases and running SQL querys within Python.
It consists of two main components, the SQLAlchemy Core and the SQLAlchemy ORM, which can be installed via pip.

The SQLAlchemy Core is the main library of SQLAlchemy and acts as the base of the toolkit. The Core provides the tools for managing and interacting with the database, queries, and query results. The Core notably contains the SQL Expression Language allows for the ability to construct and execute SQL expressions.
The ORM is an optional toolset that builds on the Core, and provides object relational mapping capabilites. The ORM allows for mapping of Python classes to SQL database tables. Using the ORM in conjuction witht the SQLAlchemy Core shifts the code to be more object oriented 

#2
SLQAlchemy is a powerful tool for working with datasets, by mapping the tables from an SQL database onto python objects, it also allows the user to selects rows from joins and other SQL queries, which is not an option in SQL. Code written for SLQAlchemy is broadly applicable to any database, unlike code written in SQLite.
The use of SQLAlchemy ORM creates a layer of abstraction over the actual database, which can cause some loses in effeciency, as well as making it easy for a user to not understand what is actually happening in the database.


In [3]:
#3
!pip install SQLAlchemy


In [5]:
import sqlalchemy
sqlalchemy.__version__  

'2.0.30'

In [ ]:
#This is just me working through the tutorial

In [6]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [7]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2024-11-17 16:31:25,019 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 16:31:25,028 INFO sqlalchemy.engine.Engine select 'hello world'
2024-11-17 16:31:25,065 INFO sqlalchemy.engine.Engine [generated in 0.04852s] ()
[('hello world',)]
2024-11-17 16:31:25,092 INFO sqlalchemy.engine.Engine ROLLBACK


In [8]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2024-11-17 16:33:06,285 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 16:33:06,289 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2024-11-17 16:33:06,293 INFO sqlalchemy.engine.Engine [generated in 0.00781s] ()
2024-11-17 16:33:06,295 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-11-17 16:33:06,295 INFO sqlalchemy.engine.Engine [generated in 0.00408s] [(1, 1), (2, 4)]
2024-11-17 16:33:06,304 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2024-11-17 16:35:09,169 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 16:35:09,177 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-11-17 16:35:09,178 INFO sqlalchemy.engine.Engine [cached since 122.9s ago] [(6, 8), (9, 10)]
2024-11-17 16:35:09,188 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-11-17 16:37:49,253 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 16:37:49,253 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2024-11-17 16:37:49,261 INFO sqlalchemy.engine.Engine [generated in 0.00961s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2024-11-17 16:37:49,261 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

2024-11-17 17:39:30,072 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 17:39:30,076 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-11-17 17:39:30,080 INFO sqlalchemy.engine.Engine [cached since 3984s ago] [(11, 12), (13, 14)]
2024-11-17 17:39:30,089 INFO sqlalchemy.engine.Engine COMMIT


In [12]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

2024-11-17 17:40:23,032 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 17:40:23,044 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-11-17 17:40:23,066 INFO sqlalchemy.engine.Engine [cached since 4037s ago] [(11, 12), (13, 14)]
2024-11-17 17:40:23,070 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-11-17 17:40:45,328 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 17:40:45,328 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2024-11-17 17:40:45,336 INFO sqlalchemy.engine.Engine [cached since 3776s ago] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
x: 11  y: 12
x: 13  y: 14
2024-11-17 17:40:45,336 INFO sqlalchemy.engine.Engine ROLLBACK


In [14]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-11-17 17:41:11,148 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 17:41:11,157 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2024-11-17 17:41:11,164 INFO sqlalchemy.engine.Engine [generated in 0.00454s] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 11  y: 12
x: 13  y: 14
x: 13  y: 14
2024-11-17 17:41:11,164 INFO sqlalchemy.engine.Engine ROLLBACK


In [15]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}],
    )
    session.commit()

2024-11-17 17:41:50,373 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 17:41:50,373 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2024-11-17 17:41:50,373 INFO sqlalchemy.engine.Engine [generated in 0.00116s] [(11, 9), (15, 13)]
2024-11-17 17:41:50,381 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [17]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [20]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [21]:
user_table.c.keys()

['id', 'name', 'fullname']

In [22]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [23]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

In [24]:
metadata_obj.create_all(engine)

2024-11-17 17:47:38,527 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 17:47:38,542 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-11-17 17:47:38,551 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-17 17:47:38,551 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-11-17 17:47:38,558 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-17 17:47:38,558 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-11-17 17:47:38,574 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-17 17:47:38,577 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-11-17 17:47:38,582 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-17 17:47:38,585 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-11-17 17:47:38,592 INFO sqlalchemy.engine.Engine [no key 0.00582s] ()
2024-11-17 17:47:38,599 INFO sqlalchemy.engine.Engine 
C

In [25]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

In [26]:
Base.metadata

MetaData()

In [27]:
Base.registry

In [28]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [29]:
#Creating a table with some data from a SQL homework assignement from another class
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE patient (PID int, FirstName text, LastName text, Gender text, Age int )"))
    conn.commit()

2024-11-17 17:58:34,598 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 17:58:34,607 INFO sqlalchemy.engine.Engine CREATE TABLE patient (PID int, FirstName text, LastName text, Gender text, Age int )
2024-11-17 17:58:34,608 INFO sqlalchemy.engine.Engine [generated in 0.00512s] ()
2024-11-17 17:58:34,612 INFO sqlalchemy.engine.Engine COMMIT


In [31]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO Patient (PID, FirstName, LastName, Gender, Age) VALUES (:PID, :FirstName, :LastName, :Gender, :Age)"),
        [{'PID': 1001, 'FirstName': 'John', 'LastName': 'Porter', 'Gender': 'Male' , 'Age': 30}, {'PID': 1002, 'FirstName': 'Jaehoon', 'LastName': 'Adams', 'Gender': 'Male' , 'Age': 15}],
    )

2024-11-17 18:05:56,895 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 18:05:56,899 INFO sqlalchemy.engine.Engine INSERT INTO Patient (PID, FirstName, LastName, Gender, Age) VALUES (?, ?, ?, ?, ?)
2024-11-17 18:05:56,902 INFO sqlalchemy.engine.Engine [generated in 0.00338s] [(1001, 'John', 'Porter', 'Male', 30), (1002, 'Jaehoon', 'Adams', 'Male', 15)]
2024-11-17 18:05:56,905 INFO sqlalchemy.engine.Engine COMMIT


In [33]:
#Printing the data from the table
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM patient"))
    for row in result:
        print(f"PID: {row.PID}  Name: {row.FirstName} {row.LastName} Gender:{row.Gender} Age:{row.Age}")

2024-11-17 18:08:46,884 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 18:08:46,884 INFO sqlalchemy.engine.Engine SELECT * FROM patient
2024-11-17 18:08:46,895 INFO sqlalchemy.engine.Engine [cached since 8.917s ago] ()
PID: 1001  Name: John Porter Gender:Male Age:30
PID: 1002  Name: Jaehoon Adams Gender:Male Age:15
2024-11-17 18:08:46,898 INFO sqlalchemy.engine.Engine ROLLBACK


In [34]:
#Selecting only two columns from the table
with engine.connect() as conn:
    result = conn.execute(text("SELECT FirstName, LastName FROM patient"))
    for row in result:
        print(f"{row.FirstName} {row.LastName} ")

2024-11-17 18:09:58,904 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-17 18:09:58,909 INFO sqlalchemy.engine.Engine SELECT FirstName, LastName FROM patient
2024-11-17 18:09:58,912 INFO sqlalchemy.engine.Engine [generated in 0.00538s] ()
John Porter 
Jaehoon Adams 
2024-11-17 18:09:58,913 INFO sqlalchemy.engine.Engine ROLLBACK
